In [1]:
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from fact.io import read_h5py
from matplotlib import pyplot as plt
plt.style.use('msackel')
%matplotlib inline

exec(open('/home/msackel/Desktop/gammaClassification/programm/theta_cut/theta_cut.py').read())
exec(open('/home/msackel/Desktop/gammaClassification/programm/model_significance/model_significance.py').read())

Tree = RandomForestClassifier(
				max_depth=15, 
				max_features=7, 
				criterion='entropy', 
				n_estimators=100, 
				n_jobs=25
				)


with open('/home/msackel/Desktop/gammaClassification/config/feature.yaml') as f:
		feature = yaml.load(f)

proton_data = pd.read_hdf('/home/msackel/Desktop/gammaClassification/data/raw_data/proton_precuts.hdf5')[feature]
proton_data['label']=0

crab_data = read_h5py(
				'/home/msackel/Desktop/gammaClassification/data/raw_data/crab_precuts.hdf5',
				key='events',
				columns=list(feature)+['theta_deg']
				)
crab_data = crab_data.query('theta_deg >='+str(np.sqrt(0.5)))
crab_data['label']=0

gamma_data = pd.read_hdf('/home/msackel/Desktop/gammaClassification/data/raw_data/gamma_precuts.hdf5')[feature]
gamma_data['label']=1

In [2]:
train_crab = pd.concat([crab_data.drop('theta_deg', axis=1)[:100000], gamma_data[:100000]])
train_mont = pd.concat([proton_data[:100000], gamma_data[:100000]])

In [3]:
Tree.fit(train_crab.drop('label', axis=1), train_crab.label)
pred_tree = Tree.predict_proba(crab_data[feature][100000:])[:,1]
pred_Tree = Tree.predict_proba(proton_data[feature][100000:])[:,1]

fig = plt.figure()
ax1 = fig.add_subplot(221)
plt.title('Conf distribution trained on Crab-pro')
plt.xlabel('conf')
plt.hist(pred_tree, bins=30, histtype='step',normed=True, label='MC')
plt.hist(pred_Tree, bins=30, histtype='step',normed=True, label='Measured')

KeyboardInterrupt: 

In [ ]:
Tree.fit(train_mont.drop('label', axis=1), train_mont.label)
pred_tree = Tree.predict_proba(crab_data[feature][100000:])[:,1]
pred_Tree = Tree.predict_proba(proton_data[feature][100000:])[:,1]

ax2 = fig.add_subplot(222, sharey=ax1)
plt.title('Conf distribution trained on MC-pro')
plt.xlabel('conf')
plt.hist(pred_tree, bins=30, histtype='step',normed=True, label='MC')
plt.hist(pred_Tree, bins=30, histtype='step',normed=True, label='Measured')
plt.setp(ax2.get_yticklabels(), visible=False) # make these tick labels invisible
fig.set_size_inches(7,5)
fig.tight_layout()
plt.legend()
plt.savefig('conf.pdf')